In [126]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle
from functools import partial
from tqdm import tqdm

from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from catboost import CatBoostRegressor
from sklearn.base import clone

In [7]:
#Сделаем настройки
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
plt.rcParams['figure.figsize'] = 12, 8
RANDOM_STATE=43

In [8]:
df = pd.read_csv('data2.csv')
df_train = df[df.is_train==1]

C:\Soft\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df.shape

(70896, 43)

# Определим нужные нам функции

In [14]:
def mape(y_true, y_pred):
    """оценка MEAN ABSOLUTE PERCENTAGE ERROR, которая используется в соревновании"""
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def score_model(model, X, y):
    """Оцениваем модель и данные на трех фолдах. Выводим средюю оценку, и оценки на фолдах"""
    kf = KFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    cv_results = cross_val_score(
        model, X, y, cv=kf, n_jobs=-1, scoring=make_scorer(mape))
    print("#"*100)
    print(model)
    print("#"*100)
    print(f'mean_result: {np.mean(cv_results)} ({cv_results})')
    print("#"*100)


def make_submit(model, X_test, submit_file='predict.csv'):
    """запишем предсказание модели в файл, готовый к отправке на kaggle"""
    pred = np.round(model.predict(X_test))
    answer = pd.Series(pred, index=df[~df.is_train].sell_id, name='price')
    answer.to_csv(submit_file, index_label='sell_id')


def prepare_data(categorical_columns: list, numerical_columns: list):
    """Вытащим требуемые данные из переменной df
    категориальные применим dummy кодирование
    числовые Шкалирование"""
    X = df.loc[:, categorical_columns+numerical_columns+['is_train']]

    X = pd.get_dummies(X, columns=categorical_columns)
    X[numerical_columns] = StandardScaler().fit_transform(X[numerical_columns])
    
    X_train = X[X.is_train == 1].drop('is_train', axis=1)
    X_test = X[X.is_train == 0].drop('is_train', axis=1)

    y = df[df.is_train].price

    return X_train, X_test, y

# Подготовим тренировочные и тестовые наборы 

Состав колонок как в предыдущей части

In [15]:
cat_cols = ['bodyType', 'brand', 'fuelType', 'color', 'model_name',
            'vehicleTransmission', 'vendor', 'ПТС', 'Владельцы', 'Привод', 'Руль'] 
num_cols = ['productionDate', 'numberOfDoors',
            'enginePower_log', 'mileage_log', 'diff_date_model_production']

In [16]:
X_train, X_test, y = prepare_data(cat_cols, num_cols)

# Подбор параметров модели

Лучший результат показал случайный лес. Давайте попробуем как будет он себя вести, при изменени его деволтных параметров  
Хотел взять критерий MAE для  модели, но она очень долго тренирутеся, даже если поставить n_estimators=10

По результатам побора параметров - выяснилось что лучше всего случайный лес работает на стандартных параметрах.

In [9]:
rf_model=RandomForestRegressor(random_state=RANDOM_STATE, n_estimators=100)

In [10]:
param_grid = { 
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,6, None],
}

In [12]:
CV_rf = GridSearchCV(rf_model, param_grid, cv=3)


In [13]:
%%time
CV_rf.fit(X_train, y)

Wall time: 11min 8s


GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=43),
             param_grid={'max_depth': [4, 6, None],
                         'max_features': ['auto', 'sqrt', 'log2']})

In [16]:
CV_rf.best_params_

{'max_depth': None, 'max_features': 'auto'}

ну т.е. дефолтные параметры оказались лучшими.  
не очень интересно

In [18]:
score_model(CV_rf.best_estimator_, X_train, y)

####################################################################################################
RandomForestRegressor(random_state=43)
####################################################################################################
mean_result: 14.034526509801124 ([13.88094322 14.38738953 13.83524678])
####################################################################################################


# Стекинг

Стекинг (stacking) — агрегация ответов моделей машинного обучения при помощи ещё одной модели машинного обучения. Подход использует понятие базовых моделей, каждая из которых обучается независимо от остальных, и мета-модели, которая использует предсказания базовых моделей как признаки.

По результатам стекинг - не улучшил модель. Может я выбрал неудачные модели для ансамбля.  
Результат и на валидации, и на лидерборде оказался хуже, чем применение одного случайного леса.  

In [122]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    X_meta_train = np.zeros((len(y_train)), dtype=np.float32)

    splits = cv.split(X_train)
    
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict(X_test)
    
    return X_meta_train, X_meta_test

def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
    
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ]).reshape(-1,len(features), order='F')

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ]).reshape(-1,len(features), order='F')
    
    return stacked_features_train, stacked_features_test

cv = KFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

In [127]:
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=RANDOM_STATE),
    ExtraTreesRegressor(n_estimators=100, random_state=RANDOM_STATE),
    Ridge(random_state=RANDOM_STATE),
    GradientBoostingRegressor(n_estimators=100, random_state=RANDOM_STATE),
    CatBoostRegressor(random_state=RANDOM_STATE)
], X_train.values, X_test.values, y.values, cv)

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [20:48<04:27, 267.55s/it]

Learning rate set to 0.06879
0:	learn: 2168828.6161248	total: 173ms	remaining: 2m 52s
1:	learn: 2066444.2719691	total: 182ms	remaining: 1m 31s
2:	learn: 1967185.9074871	total: 191ms	remaining: 1m 3s
3:	learn: 1873859.6810291	total: 201ms	remaining: 49.9s
4:	learn: 1786272.8158613	total: 209ms	remaining: 41.7s
5:	learn: 1708351.2670588	total: 218ms	remaining: 36.2s
6:	learn: 1633470.3167087	total: 228ms	remaining: 32.4s
7:	learn: 1564466.8637620	total: 237ms	remaining: 29.3s
8:	learn: 1498126.1689154	total: 245ms	remaining: 27s
9:	learn: 1438312.8829611	total: 255ms	remaining: 25.2s
10:	learn: 1378034.8491643	total: 263ms	remaining: 23.7s
11:	learn: 1328599.2286637	total: 272ms	remaining: 22.4s
12:	learn: 1280799.6997826	total: 281ms	remaining: 21.3s
13:	learn: 1235536.2875705	total: 290ms	remaining: 20.4s
14:	learn: 1193245.0454390	total: 298ms	remaining: 19.6s
15:	learn: 1154101.3576578	total: 306ms	remaining: 18.8s
16:	learn: 1117444.9844019	total: 315ms	remaining: 18.2s
17:	learn: 1

158:	learn: 497888.9810304	total: 1.6s	remaining: 8.48s
159:	learn: 497087.9293175	total: 1.61s	remaining: 8.48s
160:	learn: 495887.1993741	total: 1.63s	remaining: 8.47s
161:	learn: 494945.4815854	total: 1.63s	remaining: 8.46s
162:	learn: 494102.3723180	total: 1.64s	remaining: 8.44s
163:	learn: 493915.5868024	total: 1.65s	remaining: 8.43s
164:	learn: 493711.9972959	total: 1.66s	remaining: 8.41s
165:	learn: 492653.2812184	total: 1.67s	remaining: 8.39s
166:	learn: 491646.6640032	total: 1.68s	remaining: 8.37s
167:	learn: 491495.5599526	total: 1.69s	remaining: 8.35s
168:	learn: 490319.2698092	total: 1.69s	remaining: 8.33s
169:	learn: 488510.7285476	total: 1.7s	remaining: 8.31s
170:	learn: 487494.2590107	total: 1.71s	remaining: 8.3s
171:	learn: 486520.0181201	total: 1.72s	remaining: 8.28s
172:	learn: 486369.3258383	total: 1.73s	remaining: 8.26s
173:	learn: 486203.5218114	total: 1.74s	remaining: 8.24s
174:	learn: 486064.7991087	total: 1.74s	remaining: 8.22s
175:	learn: 484858.8608209	total: 

313:	learn: 405661.8950166	total: 2.96s	remaining: 6.47s
314:	learn: 405089.4759842	total: 2.97s	remaining: 6.46s
315:	learn: 404631.6782069	total: 2.98s	remaining: 6.45s
316:	learn: 404189.5131785	total: 2.99s	remaining: 6.44s
317:	learn: 404102.1037130	total: 3s	remaining: 6.43s
318:	learn: 403501.4942124	total: 3.01s	remaining: 6.42s
319:	learn: 403019.7060412	total: 3.02s	remaining: 6.41s
320:	learn: 402681.6348029	total: 3.02s	remaining: 6.4s
321:	learn: 402283.3640314	total: 3.03s	remaining: 6.39s
322:	learn: 401710.5216010	total: 3.04s	remaining: 6.38s
323:	learn: 401561.6207979	total: 3.05s	remaining: 6.37s
324:	learn: 401074.6023321	total: 3.06s	remaining: 6.36s
325:	learn: 400598.6214949	total: 3.07s	remaining: 6.35s
326:	learn: 400157.1693012	total: 3.08s	remaining: 6.34s
327:	learn: 399990.7274522	total: 3.09s	remaining: 6.32s
328:	learn: 399923.9660182	total: 3.09s	remaining: 6.31s
329:	learn: 399596.1257919	total: 3.1s	remaining: 6.3s
330:	learn: 399352.5539910	total: 3.1

472:	learn: 360741.3439899	total: 4.33s	remaining: 4.83s
473:	learn: 360509.4035423	total: 4.35s	remaining: 4.82s
474:	learn: 359903.4448607	total: 4.36s	remaining: 4.81s
475:	learn: 359529.3620262	total: 4.36s	remaining: 4.8s
476:	learn: 359439.0915042	total: 4.37s	remaining: 4.79s
477:	learn: 359311.2093982	total: 4.38s	remaining: 4.78s
478:	learn: 359180.8856533	total: 4.39s	remaining: 4.77s
479:	learn: 359088.6750330	total: 4.4s	remaining: 4.76s
480:	learn: 359046.7647853	total: 4.4s	remaining: 4.75s
481:	learn: 359009.8773738	total: 4.41s	remaining: 4.74s
482:	learn: 358807.3544935	total: 4.42s	remaining: 4.73s
483:	learn: 358673.4835550	total: 4.43s	remaining: 4.72s
484:	learn: 358377.1304290	total: 4.44s	remaining: 4.71s
485:	learn: 358063.7128442	total: 4.45s	remaining: 4.7s
486:	learn: 357813.5381931	total: 4.45s	remaining: 4.69s
487:	learn: 357420.8428617	total: 4.46s	remaining: 4.68s
488:	learn: 357361.3497117	total: 4.47s	remaining: 4.67s
489:	learn: 357170.1676962	total: 4

...

514:	learn: 357535.5538384	total: 6.03s	remaining: 5.68s
515:	learn: 357280.3446220	total: 6.04s	remaining: 5.66s
516:	learn: 357039.0802578	total: 6.05s	remaining: 5.65s
517:	learn: 356968.6414888	total: 6.06s	remaining: 5.64s
518:	learn: 356860.1399464	total: 6.08s	remaining: 5.63s
519:	learn: 356647.3312012	total: 6.09s	remaining: 5.63s
520:	learn: 356616.0774628	total: 6.11s	remaining: 5.61s
521:	learn: 356166.2354580	total: 6.12s	remaining: 5.61s
522:	learn: 355860.3969891	total: 6.14s	remaining: 5.6s
523:	learn: 355511.2187761	total: 6.15s	remaining: 5.59s
524:	learn: 355444.8451682	total: 6.17s	remaining: 5.58s
525:	learn: 355220.6364206	total: 6.18s	remaining: 5.57s
526:	learn: 354862.3828591	total: 6.19s	remaining: 5.56s
527:	learn: 354664.0314617	total: 6.21s	remaining: 5.55s
528:	learn: 354393.8790253	total: 6.22s	remaining: 5.54s
529:	learn: 354197.9358291	total: 6.24s	remaining: 5.53s
530:	learn: 354131.9404838	total: 6.25s	remaining: 5.52s
531:	learn: 353844.5615725	total

664:	learn: 332153.7136324	total: 8.19s	remaining: 4.13s
665:	learn: 331984.9487634	total: 8.21s	remaining: 4.12s
666:	learn: 331805.6491294	total: 8.23s	remaining: 4.11s
667:	learn: 331770.7832558	total: 8.24s	remaining: 4.09s
668:	learn: 331476.1929198	total: 8.25s	remaining: 4.08s
669:	learn: 331442.9715845	total: 8.26s	remaining: 4.07s
670:	learn: 331271.7054522	total: 8.27s	remaining: 4.06s
671:	learn: 331172.8839689	total: 8.29s	remaining: 4.04s
672:	learn: 331146.8832506	total: 8.3s	remaining: 4.03s
673:	learn: 330910.2263155	total: 8.31s	remaining: 4.02s
674:	learn: 330783.0491740	total: 8.32s	remaining: 4.01s
675:	learn: 330750.6845807	total: 8.33s	remaining: 3.99s
676:	learn: 330506.2154642	total: 8.36s	remaining: 3.99s
677:	learn: 330356.2636666	total: 8.38s	remaining: 3.98s
678:	learn: 330201.5551423	total: 8.39s	remaining: 3.97s
679:	learn: 330034.6260284	total: 8.41s	remaining: 3.96s
680:	learn: 329842.4152109	total: 8.42s	remaining: 3.95s
681:	learn: 329727.8458906	total

816:	learn: 313288.3214270	total: 10.4s	remaining: 2.32s
817:	learn: 313196.2469462	total: 10.4s	remaining: 2.31s
818:	learn: 313047.5128443	total: 10.4s	remaining: 2.3s
819:	learn: 312916.4259213	total: 10.4s	remaining: 2.28s
820:	learn: 312866.0109330	total: 10.4s	remaining: 2.28s
821:	learn: 312702.0185531	total: 10.5s	remaining: 2.26s
822:	learn: 312687.8737376	total: 10.5s	remaining: 2.25s
823:	learn: 312562.6482602	total: 10.5s	remaining: 2.24s
824:	learn: 312426.5847825	total: 10.5s	remaining: 2.23s
825:	learn: 312370.8377388	total: 10.5s	remaining: 2.21s
826:	learn: 312357.0597859	total: 10.5s	remaining: 2.21s
827:	learn: 312313.8912238	total: 10.6s	remaining: 2.19s
828:	learn: 312242.0009822	total: 10.6s	remaining: 2.18s
829:	learn: 312207.8710961	total: 10.6s	remaining: 2.17s
830:	learn: 312043.5194494	total: 10.6s	remaining: 2.16s
831:	learn: 311860.9811528	total: 10.6s	remaining: 2.15s
832:	learn: 311847.6210426	total: 10.6s	remaining: 2.13s
833:	learn: 311739.1011936	total

970:	learn: 299746.8244300	total: 12.5s	remaining: 373ms
971:	learn: 299650.6158150	total: 12.5s	remaining: 360ms
972:	learn: 299546.7858955	total: 12.5s	remaining: 347ms
973:	learn: 299471.6174256	total: 12.5s	remaining: 334ms
974:	learn: 299417.0981786	total: 12.5s	remaining: 321ms
975:	learn: 299407.1751723	total: 12.5s	remaining: 308ms
976:	learn: 299313.1831339	total: 12.5s	remaining: 295ms
977:	learn: 299178.6719603	total: 12.6s	remaining: 283ms
978:	learn: 299094.2970155	total: 12.6s	remaining: 270ms
979:	learn: 299068.5805284	total: 12.6s	remaining: 257ms
980:	learn: 299021.9051555	total: 12.6s	remaining: 244ms
981:	learn: 298966.0542473	total: 12.6s	remaining: 231ms
982:	learn: 298881.9192200	total: 12.6s	remaining: 218ms
983:	learn: 298826.2736170	total: 12.6s	remaining: 205ms
984:	learn: 298749.8827603	total: 12.6s	remaining: 192ms
985:	learn: 298690.6271942	total: 12.6s	remaining: 179ms
986:	learn: 298621.3368332	total: 12.7s	remaining: 167ms
987:	learn: 298533.1930866	tota

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [22:04<00:00, 264.85s/it]


In [128]:
lr = LinearRegression()
lr.fit(stacked_features_train, y)

LinearRegression()

In [129]:
mape(y, lr.predict(stacked_features_train))

14.317001402378082

In [130]:
make_submit(lr, stacked_features_test) # 16.47279

Плохой результат получался, увеличим количество split до 10

In [133]:
cv = KFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

In [134]:
stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=RANDOM_STATE),
    Ridge(random_state=RANDOM_STATE),
    GradientBoostingRegressor(n_estimators=100, random_state=RANDOM_STATE),
    CatBoostRegressor(random_state=RANDOM_STATE)
], X_train.values, X_test.values, y.values, cv)


 75%|██████████████████████████████████████████████████████████████▎                    | 3/4 [31:42<11:49, 709.70s/it]

Learning rate set to 0.072565
0:	learn: 2243244.6776906	total: 11.2ms	remaining: 11.2s
1:	learn: 2129729.1905009	total: 21.8ms	remaining: 10.9s
2:	learn: 2025224.3475044	total: 32.2ms	remaining: 10.7s
3:	learn: 1929758.5851419	total: 42ms	remaining: 10.5s
4:	learn: 1838117.8735264	total: 51.9ms	remaining: 10.3s
5:	learn: 1754347.8798048	total: 61.4ms	remaining: 10.2s
6:	learn: 1676480.9077490	total: 70.8ms	remaining: 10s
7:	learn: 1603135.7420766	total: 80.2ms	remaining: 9.94s
8:	learn: 1538943.8508341	total: 89.7ms	remaining: 9.87s
9:	learn: 1476745.9947426	total: 99.2ms	remaining: 9.82s
10:	learn: 1419455.3999543	total: 109ms	remaining: 9.79s
11:	learn: 1364378.2176085	total: 119ms	remaining: 9.79s
12:	learn: 1313719.5263037	total: 128ms	remaining: 9.75s
13:	learn: 1267294.4513996	total: 138ms	remaining: 9.71s
14:	learn: 1226202.8956660	total: 149ms	remaining: 9.81s
15:	learn: 1187577.4335476	total: 165ms	remaining: 10.1s
16:	learn: 1147861.1283538	total: 176ms	remaining: 10.2s
17:	l

155:	learn: 515905.0713657	total: 1.57s	remaining: 8.52s
156:	learn: 515546.8628784	total: 1.59s	remaining: 8.52s
157:	learn: 513736.0117606	total: 1.6s	remaining: 8.51s
158:	learn: 513404.5484347	total: 1.61s	remaining: 8.5s
159:	learn: 513148.0887384	total: 1.62s	remaining: 8.49s
160:	learn: 512853.8258137	total: 1.63s	remaining: 8.47s
161:	learn: 511327.8726523	total: 1.64s	remaining: 8.46s
162:	learn: 510265.1686731	total: 1.65s	remaining: 8.45s
163:	learn: 509595.9648012	total: 1.66s	remaining: 8.44s
164:	learn: 508457.7277006	total: 1.67s	remaining: 8.43s
165:	learn: 506709.4688493	total: 1.68s	remaining: 8.42s
166:	learn: 505428.6238443	total: 1.68s	remaining: 8.4s
167:	learn: 503851.9526880	total: 1.69s	remaining: 8.39s
168:	learn: 502427.5974011	total: 1.7s	remaining: 8.38s
169:	learn: 502192.7344041	total: 1.71s	remaining: 8.36s
170:	learn: 500973.2879690	total: 1.72s	remaining: 8.35s
171:	learn: 499609.3182041	total: 1.73s	remaining: 8.34s
172:	learn: 499281.4910987	total: 1

306:	learn: 417416.2195132	total: 3.12s	remaining: 7.03s
307:	learn: 416846.2931181	total: 3.13s	remaining: 7.02s
308:	learn: 416644.1908906	total: 3.13s	remaining: 7.01s
309:	learn: 416121.6730506	total: 3.15s	remaining: 7s
310:	learn: 415753.9099958	total: 3.15s	remaining: 6.99s
311:	learn: 415150.7736490	total: 3.16s	remaining: 6.98s
312:	learn: 414672.6526763	total: 3.17s	remaining: 6.97s
313:	learn: 414050.3448280	total: 3.18s	remaining: 6.96s
314:	learn: 413781.1824397	total: 3.19s	remaining: 6.95s
315:	learn: 413002.8852376	total: 3.2s	remaining: 6.94s
316:	learn: 412277.2026559	total: 3.21s	remaining: 6.92s
317:	learn: 412171.8221440	total: 3.22s	remaining: 6.91s
318:	learn: 411761.8108939	total: 3.23s	remaining: 6.91s
319:	learn: 411415.2733806	total: 3.25s	remaining: 6.89s
320:	learn: 411335.1654014	total: 3.25s	remaining: 6.88s
321:	learn: 411057.6335005	total: 3.26s	remaining: 6.87s
322:	learn: 410971.5005747	total: 3.27s	remaining: 6.86s
323:	learn: 410453.3397318	total: 3

467:	learn: 368664.1247122	total: 4.73s	remaining: 5.38s
468:	learn: 368479.5021513	total: 4.75s	remaining: 5.37s
469:	learn: 368268.7237355	total: 4.76s	remaining: 5.36s
470:	learn: 368074.7115676	total: 4.77s	remaining: 5.36s
471:	learn: 367630.7897541	total: 4.78s	remaining: 5.35s
472:	learn: 367403.5459779	total: 4.79s	remaining: 5.34s
473:	learn: 367154.8019272	total: 4.8s	remaining: 5.33s
474:	learn: 367113.0979346	total: 4.81s	remaining: 5.31s
475:	learn: 366828.8506579	total: 4.82s	remaining: 5.3s
476:	learn: 366566.8207085	total: 4.83s	remaining: 5.29s
477:	learn: 366510.1475108	total: 4.84s	remaining: 5.28s
478:	learn: 366261.2929616	total: 4.85s	remaining: 5.27s
479:	learn: 366054.2856448	total: 4.86s	remaining: 5.26s
480:	learn: 365543.4316449	total: 4.87s	remaining: 5.25s
481:	learn: 365412.7687889	total: 4.88s	remaining: 5.24s
482:	learn: 365168.7440521	total: 4.88s	remaining: 5.23s
483:	learn: 365128.3643143	total: 4.89s	remaining: 5.22s
484:	learn: 364675.7924689	total:

...

495:	learn: 361772.7801497	total: 5.29s	remaining: 5.38s
496:	learn: 361603.8231115	total: 5.3s	remaining: 5.37s
497:	learn: 361481.3703782	total: 5.31s	remaining: 5.36s
498:	learn: 361056.4918124	total: 5.33s	remaining: 5.35s
499:	learn: 360830.6864332	total: 5.34s	remaining: 5.34s
500:	learn: 360532.2198822	total: 5.35s	remaining: 5.32s
501:	learn: 360006.4299631	total: 5.36s	remaining: 5.31s
502:	learn: 359973.4355750	total: 5.37s	remaining: 5.3s
503:	learn: 359898.5155334	total: 5.38s	remaining: 5.29s
504:	learn: 359645.9211835	total: 5.38s	remaining: 5.28s
505:	learn: 359167.2009634	total: 5.4s	remaining: 5.27s
506:	learn: 358752.3675142	total: 5.41s	remaining: 5.26s
507:	learn: 358629.9462184	total: 5.42s	remaining: 5.25s
508:	learn: 358528.9680229	total: 5.43s	remaining: 5.24s
509:	learn: 358374.5542087	total: 5.44s	remaining: 5.22s
510:	learn: 358342.3892510	total: 5.45s	remaining: 5.21s
511:	learn: 358224.7685469	total: 5.46s	remaining: 5.2s
512:	learn: 358028.2170864	total: 5

644:	learn: 334422.8656644	total: 6.87s	remaining: 3.78s
645:	learn: 334235.8018942	total: 6.88s	remaining: 3.77s
646:	learn: 334067.8994921	total: 6.89s	remaining: 3.76s
647:	learn: 333914.0932494	total: 6.91s	remaining: 3.75s
648:	learn: 333808.1954294	total: 6.92s	remaining: 3.74s
649:	learn: 333786.2129357	total: 6.92s	remaining: 3.73s
650:	learn: 333598.9627991	total: 6.93s	remaining: 3.72s
651:	learn: 333561.4629675	total: 6.95s	remaining: 3.71s
652:	learn: 333316.0580549	total: 6.96s	remaining: 3.7s
653:	learn: 333282.7998580	total: 6.96s	remaining: 3.69s
654:	learn: 333154.8911942	total: 6.97s	remaining: 3.67s
655:	learn: 332913.9694800	total: 6.99s	remaining: 3.66s
656:	learn: 332891.3864553	total: 7s	remaining: 3.65s
657:	learn: 332869.5231218	total: 7s	remaining: 3.64s
658:	learn: 332848.3701837	total: 7.01s	remaining: 3.63s
659:	learn: 332589.6207765	total: 7.02s	remaining: 3.62s
660:	learn: 332569.0647813	total: 7.03s	remaining: 3.61s
661:	learn: 332549.0631584	total: 7.04

792:	learn: 315659.1602214	total: 8.44s	remaining: 2.2s
793:	learn: 315581.4093483	total: 8.45s	remaining: 2.19s
794:	learn: 315452.1251686	total: 8.46s	remaining: 2.18s
795:	learn: 315332.2408958	total: 8.47s	remaining: 2.17s
796:	learn: 315256.1929248	total: 8.48s	remaining: 2.16s
797:	learn: 315193.4089847	total: 8.49s	remaining: 2.15s
798:	learn: 315106.5779160	total: 8.5s	remaining: 2.14s
799:	learn: 315060.7353547	total: 8.51s	remaining: 2.13s
800:	learn: 315045.7414808	total: 8.52s	remaining: 2.12s
801:	learn: 314857.3872467	total: 8.53s	remaining: 2.1s
802:	learn: 314779.5720126	total: 8.54s	remaining: 2.1s
803:	learn: 314669.3344112	total: 8.55s	remaining: 2.08s
804:	learn: 314654.7922685	total: 8.56s	remaining: 2.07s
805:	learn: 314554.4486633	total: 8.57s	remaining: 2.06s
806:	learn: 314483.7097498	total: 8.58s	remaining: 2.05s
807:	learn: 314405.7307702	total: 8.6s	remaining: 2.04s
808:	learn: 314363.0033707	total: 8.61s	remaining: 2.03s
809:	learn: 314235.3489178	total: 8.

942:	learn: 302039.1606678	total: 10s	remaining: 607ms
943:	learn: 301957.1796672	total: 10.1s	remaining: 597ms
944:	learn: 301907.7352108	total: 10.1s	remaining: 586ms
945:	learn: 301867.1352887	total: 10.1s	remaining: 575ms
946:	learn: 301770.9117443	total: 10.1s	remaining: 565ms
947:	learn: 301671.5408535	total: 10.1s	remaining: 554ms
948:	learn: 301557.7972731	total: 10.1s	remaining: 543ms
949:	learn: 301440.4662706	total: 10.1s	remaining: 533ms
950:	learn: 301360.0058733	total: 10.1s	remaining: 522ms
951:	learn: 301250.1671919	total: 10.1s	remaining: 511ms
952:	learn: 301239.9255975	total: 10.2s	remaining: 501ms
953:	learn: 301022.1035766	total: 10.2s	remaining: 490ms
954:	learn: 300978.3348332	total: 10.2s	remaining: 479ms
955:	learn: 300892.7289198	total: 10.2s	remaining: 469ms
956:	learn: 300801.4935082	total: 10.2s	remaining: 458ms
957:	learn: 300791.4332837	total: 10.2s	remaining: 447ms
958:	learn: 300721.5295066	total: 10.2s	remaining: 437ms
959:	learn: 300668.7354006	total:


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [35:22<00:00, 530.57s/it]


In [135]:
lr = LinearRegression()
lr.fit(stacked_features_train, y)

LinearRegression()

In [136]:
mape(y, lr.predict(stacked_features_train))

14.693905413150787

In [137]:
make_submit(lr, stacked_features_test) # 18.35051
